# Create SSH Tunnels

 

## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

## Create the Experiment Slice



In [2]:
# Create a slice
slice = fablib.new_slice(name="MySlice10")

# Add a node
node = slice.add_node(name="Node1", disk=100, image='docker_rocky_8')

node.add_post_boot_upload_directory('node_tools','.')
node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }}')


# Submit the slice
slice.submit();


Retry: 7, Time: 182 sec


ID,a6045c67-327d-43d3-8d48-fc84adfbbbe8
Name,MySlice10
Lease Expiration (UTC),2023-04-20 16:44:32 +0000
Lease Start (UTC),2023-04-19 16:44:33 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
79092057-a250-44f0-9398-003558307ade,Node1,2,8,100,docker_rocky_8,qcow2,ncsa-w3.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fed9:f2d8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fed9:f2d8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Run a Web Service on the Node 

Many FABRIC users want an easy way to transfer files to/from their FABRIC nodes.  [File Browser](https://filebrowser.org/) is a self-hosted webservice for transfering files.

The following cell will start a Docker container on you node that serves the File Browser webservice on port 5555.  

After running this cell, you can use a browser to connect the File Brower and transfer files to/from the node. However, the node is securely protected by the FABRIC's bastion host.  

In [ ]:
node = slice.get_node('Node1')

node.execute("docker run -d "
                "--name filebrowser "
                "-p 127.0.0.1:5555:5555 "
                f"-v /home/{node.get_username()}/data:/data "
                "-e FB_BASEURL=/filebrowser "
                "-e FB_ROOT=/data "
                "-e FB_PORT=5555 "
                "-e FB_NOAUTH=noauth "
                "filebrowser/filebrowser "
                , quiet=True, output_file=f"{node.get_name()}.log");


## Start the SSH Tunnel

- Download your custom `fabric_ssh_tunnel_tools.tgz` tarball from the main jupyter-examples folder OR the fabric_config folder.  This tarball was created when you ran the configure environment notebook. Rerun the configure environment notebook if you do not have fabric_ssh_tunnel_toold.tgz 
- Untar the tarball and put the resulting folder (`fabric_ssh_tunnel_tools`) somewhere you can access it from the command line.
- Open a terminal window. (Windows: use `powershell`) 
- Use `cd` to navigate to the `fabric_ssh_tunnel_tools` folder.
- In your terminal, run the command that results from running the following cell (leave the terminal window open).

In [ ]:
# Port on your local machine that you want to map the File Browser to.
local_port='5555'
# Local interface to map the File Browser to (can be `localhost`)
local_host='127.0.0.1'

# Port on the node used by the File Browser Service
target_port='5555'

# Username/node on FABRIC
target_host=f'{node.get_username()}@{node.get_management_ip()}'

print(f'ssh  -L {local_host}:{local_port}:127.0.0.1:{target_port} -i slice_key -F ssh_config {target_host}')

## Connect to the File Browser

The File Browser service on the node is now mapped to 127.0.0.1:5555 on your local machine. You can open a browser and navigate to the following address (or just click the link): 

[http://127.0.0.1:5555](http://127.0.0.1:5555)

The default login/password is admin/admin. If you want to use File Browser in your FABRIC experiments, please pick better passwords.

You can now drag/drop files from your laptop to the FABRIC node. By default, the files will be stored in `~/data`.  

SSH to the node and observe the files you put there.

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()